# Big Data Dashboard

In [17]:
import pymongo
import pandas as pd
import seaborn as sns
from ipywidgets import interact

sns.set(style="darkgrid")

In [18]:
DATABASE_NAME = "big_data"
STAT_TABLE = "statistics"
SLOT_TABLE = "timeslots"
WINDOW_TABLE = "window"
SENSOR_TABLE = "space_sensors"
MUNICIPALITY_TABLE = "municipality_spaces"

In [19]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017")
used_db = mongo_client[DATABASE_NAME]

statistics = used_db[STAT_TABLE]
timeslots = used_db[SLOT_TABLE]
window = used_db[WINDOW_TABLE]
sensors = used_db[SENSOR_TABLE]
municipalities = used_db[MUNICIPALITY_TABLE]

## Basic statistics

In [20]:
from datetime import *
import time
import matplotlib.pyplot as plt
import numpy as np

Please select the type, space and time grannularity of the data you would like to consult:

In [25]:
day = timedelta(days=1)

MEASUREMENT = "measurement"
MUNICIPALITY = "municipality"

# query 1: stats (min, max, avg) per type (hue 1) per space (hue 2) per time (hue 3)
@interact(sensor_type = [("temperature", 0), ("humidity",1), ("light",2), ("movement",3)], 
          space_type = ["space", "municipality", "Brussels"],
          period = [("24hrs", day), ("2 days", 2 * day), ("last week", 7 * day), ("last month", 30*day), ("last year", 365*day)])

def report_statistics(sensor_type, space_type, period):
    
    if sensor_type == 0:
        reading_type = "Temperature"
    elif sensor_type == 1:
        reading_type = "Humidity"
    elif sensor_type == 2:
        reading_type = "Light"
    else:
        reading_type = "Movement"
    # define current date
    # as the last date recorded in the database
    current_date = statistics.find_one(sort=[("time", -1)])["time"]
    
    # query to obtain sensor type between current_time - period to now
    my_query = statistics.find({"sensor_type": sensor_type, "time": {'$gt': current_date - period}})
    res = pd.DataFrame([elem for elem in my_query])
    
    if not res.empty:
        # transform information in DF to solely about IDs
        res["p-i"] = res["p-i"].map(lambda x: int(x.rstrip('0123').rstrip("-")))
    
        # case: space
        if space_type == "space":

            # obtain sensors in spaces, merge with sensor information
            space_query = sensors.find()
            sp_res = pd.DataFrame([elem for elem in space_query])
            res = pd.merge(res, sp_res, on=["p-i"])

            # find max, min, avg per space
            list_max = res.groupby(["space_id"])[MEASUREMENT].max()
            list_min = res.groupby(["space_id"])[MEASUREMENT].min()
            list_mean = res.groupby(["space_id"])[MEASUREMENT].mean()

            # extract associated labels
            max_values, min_values, mean_values = list(list_max.values), list(list_min.values), list(list_mean.values)
            spaces = list(res.groupby(["space_id"]).groups.keys())

            # string for labels
            for i in range(len(spaces)):
                spaces[i] = str(spaces[i])

            plt.title(reading_type + " readings statistics for each space")
            # (max) fixed number -> can be plotted
            plt.xlabel("spaces")
            plt.ylabel("reading")
            plt.plot(spaces, max_values, 'ro')
            plt.plot(spaces, min_values, 'bo')
            plt.plot(spaces, mean_values, 'yo')
            plt.show()

            # format
            print("space\tmax\t\t\tmin\t\t\tavg")
            for i in range(len(spaces)):
                print("{}\t{}\t{}\t{}".format(spaces[i], max_values[i], min_values[i], mean_values[i]))


        # case : municipality
        elif space_type == "municipality":

            # obtain sensors in spaces, merge with sensor information
            space_query = sensors.find()
            sp_res = pd.DataFrame([elem for elem in space_query])
            res = pd.merge(res, sp_res, on=["p-i"])
            #print(res)
            #res = res.drop(["municipality"], axis=1)

            # obtain municipality information
            mun_query = municipalities.find()
            mun_res = pd.DataFrame([elem for elem in mun_query])

            # merge on space id
            res = pd.merge(res, mun_res, on=["space_id"])

            # find max, min, avg
            list_max = res.groupby([MUNICIPALITY])[MEASUREMENT].max()
            list_min = res.groupby([MUNICIPALITY])[MEASUREMENT].min()
            list_mean = res.groupby([MUNICIPALITY])[MEASUREMENT].mean()

            # extract labels
            active_mun = list(res.groupby([MUNICIPALITY]).groups.keys())
            max_values, min_values, mean_values = list(list_max.values), list(list_min.values), list(list_mean.values)

            # string for labels
            for i in range(len(active_mun)):
                active_mun[i] = str(active_mun[i])

            plt.title(reading_type + " readings statistics for the each municipality")
            # (max) fixed number -> can be plotted
            plt.xlabel("municipalities")
            plt.xticks(rotation=90)
            plt.ylabel("reading")
            plt.plot(active_mun, max_values, 'ro')
            plt.plot(active_mun, min_values, 'bo')
            plt.plot(active_mun, mean_values, 'yo')
            plt.show()

            print("municipality\tmax\t\t\tmin\t\t\tavg")
            for i in range(len(active_mun)):
                print("{}\t\t{}\t{}\t{}".format(active_mun[i], max_values[i], min_values[i], mean_values[i]))

        # case : Brussels  
        else:

            maximum_measurement = res[MEASUREMENT].max()
            minimum_measurement = res[MEASUREMENT].min()
            average_measurement = res[MEASUREMENT].mean()

            plt.title(reading_type + " readings statistics for the entirety of Brussels")
            # (max) fixed number -> can be plotted
            plt.ylabel("reading")
            plt.plot(["Brussels"], [maximum_measurement], 'ro')
            plt.plot(["Brussels"], [minimum_measurement], 'bo')
            plt.plot(["Brussels"], [average_measurement], 'yo')
            plt.show()

            print("maximum value : {}".format(maximum_measurement))
            print("minumum value : {}".format(minimum_measurement))
            print("avgerage value : {}".format(average_measurement))
    else:
        print("Unfortunately, there is no information about {} sensors..".format(reading_type))

interactive(children=(Dropdown(description='sensor_type', options=(('temperature', 0), ('humidity', 1), ('ligh…

## Timeslots

Please select the space grannularity, its privacy setting as well as time grannularity to visualize:

In [11]:
DAYTIME = "daytime"
NIGHTTIME = "nighttime"
TRESHOLD = 19.5

# query 2: for each temp sensor -> characterization timeslots
@interact(space = ["space", "municipality", "Brussels"],
          privacy = ["private", "public"],
          period = [("last month", 30*day), ("last year", 365*day)])

def classifyTimeslots(space, privacy, period):
    # define current date
    # as the last date recorded in the database
    current_date = timeslots.find_one(sort=[("time", -1)])["time"]
    
    # obtain entries in time period with specific place
    my_query = timeslots.find({"time": {'$gt': current_date - period}})
    res = pd.DataFrame([elem for elem in my_query])
    
    # transform information in DF to solely about IDs
    res["p-i"] = res["p-i"].map(lambda x: int(x.rstrip('0123').rstrip("-")))
    
    # obtain sensors in spaces, merge with sensor information
    space_query = sensors.find()
    sp_res = pd.DataFrame([elem for elem in space_query])

    res = pd.merge(res, sp_res, on=["p-i"])

    # obtain municipalities
    mun_query = municipalities.find({"privacy": privacy})
    mun_res = pd.DataFrame([elem for elem in mun_query])
    
    available_slots = timeslots.distinct("slot")  #Each different slot that has been seen -> will increase to 96
    for i in range(len(available_slots)):
        available_slots[i] = str(available_slots[i])
    # case : per space
    if space == "space":
        if not mun_res.empty:
            spaces = list(res.groupby(["space_id"]).groups.keys())
            res = pd.merge(res, mun_res, on="space_id")
            spaces_for_slots = [spaces for i in range (len(available_slots))]
            # gather measures
            slot_characterization = res.groupby(["space_id", "slot"])[MEASUREMENT]
            classif_all_slots = [[]for i in range(len(spaces))]
            # for each sensor/slot pair and readings
            #print("space\tslot\ttemperature type")
            for space_slot, readings in slot_characterization:
                space, slot = space_slot[0], space_slot[1]
                type_temp = DAYTIME
                if readings.mean() < TRESHOLD:
                    type_temp = NIGHTTIME
                space_index = spaces.index(space)
                classif_all_slots[space_index].append(type_temp)
                #print("{}\t{}\t{}".format(space, slot, type_temp))
            
            # string for labels
            for i in range(len(spaces)):
                spaces[i] = str(spaces[i])
                
            
            colors_for_space = [[]for i in range(len(spaces))]
            for i in range (len(classif_all_slots)):
                for j in range (len(classif_all_slots[i])):
                    if classif_all_slots[i][j] == NIGHTTIME:
                        colors_for_space[i].append("blue")
                    else:
                        colors_for_space[i].append("red")
                        
            plt.title("Time slots classification for each space")  
            plt.xlabel("slots")
            plt.ylabel("spaces")
            for i in range (len(spaces)):
                plt.scatter(available_slots, spaces_for_slots[i], c=colors_for_space[i])
            plt.show()
        else:
            print("No data on spaces of {} type exist ...".format(privacy))
        
        
    # case : per municipality
    elif space == "municipality":
        if not mun_res.empty:
            res = pd.merge(res, mun_res, on="space_id")
            municipality_characterization = res.groupby(["municipality", "slot"])[MEASUREMENT]
            municipality_names = list(res.groupby(["municipality"]).groups.keys())
            #print("municipality\tslot\ttemperature type")
            mun_for_slots = [municipality_names for i in range (len(available_slots))]
            classif_all_slots = [[]for i in range(len(municipality_names))]
            
            for sensor_slot, readings in municipality_characterization:
                mun, slot = sensor_slot[0], sensor_slot[1]
                type_temp = DAYTIME
                if readings.mean() < TRESHOLD:
                    type_temp = NIGHTTIME
                mun_index = municipality_names.index(mun)
                classif_all_slots[mun_index].append(type_temp)
                #print("{}\t\t{}\t{}".format(mun, slot, type_temp))
            
            # string for labels
            
            for i in range(len(municipality_names)):
                municipality_names[i] = str(municipality_names[i])
                
            colors_for_mun = [[]for i in range(len(municipality_names))]
            for i in range (len(classif_all_slots)):
                for j in range (len(classif_all_slots[i])):
                    if classif_all_slots[i][j] == NIGHTTIME:
                        colors_for_mun[i].append("blue")
                    else:
                        colors_for_mun[i].append("red")
                   
            plt.title("Time slots classification for each municipality")    
            plt.xlabel("slots")
            plt.ylabel("municipalities")
            for i in range (len(municipality_names)):
                plt.scatter(available_slots, mun_for_slots[i], c=colors_for_mun[i])
            plt.show()
        else:
            print("No data on municipalities containing spaces of {} type exist ...".format(privacy))
    
    # case : Brussels
    else:
        # not empty
        if not mun_res.empty:
            res = pd.merge(res, mun_res, on="space_id")
            classif = []
            bxl_characterization = res.groupby(["slot"])[MEASUREMENT]
            for slot, readings in bxl_characterization:
                type_temp = DAYTIME
                if readings.mean() < TRESHOLD:
                    type_temp = NIGHTTIME
                classif.append(type_temp)
            
            # string for labels
            city_name = ["Brussels" for i in range (len(available_slots))]
            
            plt.title("Time slots classification for the entirety of Brussels")    
            plt.xlabel("slots")
            plt.ylabel("municipalities")
            colors = []
            for i in range (len(classif)):
                if classif[i] == NIGHTTIME:
                    colors.append("blue")
                else:
                    colors.append("red")

            plt.scatter(available_slots, city_name, c=colors)
            plt.show()
            
            print("Brussels temperature type : {}".format(type_temp))
        else:
            print("Brussels does not contain spaces of {} type ...".format(privacy))

interactive(children=(Dropdown(description='space', options=('space', 'municipality', 'Brussels'), value='spac…

## Sliding window

Here is an overview of the most frequent measurements in the latest hour:

In [16]:
@interact(update=["update"])

def slidingWindow(update):
    hour = timedelta(hours=1)

    # define current date
    # as the last date recorded in the database
    current_date = window.find_one(sort=[("time", -1)])["time"]
    # The "window" operation has already been taken into account
    # Every record that has the current_date as its "time" value is in the last 24h window
    my_query = window.find({"time": current_date}) 
    res = pd.DataFrame([elem for elem in my_query])
    

    measures = []
    frequencies = []
    information = res.groupby([MEASUREMENT])["frequency"]
    for measure, frequency in information:
        measures.append(measure)
        frequencies.append(float(frequency.tail(1)))
        #print("{}\t{}".format(measure, float(frequency.tail(1))))
        
    for i in range (len(measures)):
        measures[i] = str(measures[i])
    x = [i for i in range (len(measures))]

    plt.title("Frequent temperature measures within the last 24h window")
    plt.ylabel("Frequency")
    plt.xlabel("Measurement")
    plt.xticks(x, measures)
    for i in range (len(measures)):
        plt.bar(x[i], frequencies[i])

    plt.show()
    

interactive(children=(Dropdown(description='update', options=('update',), value='update'), Output()), _dom_cla…